In [2]:
!git clone https://github.com/PavanDaniele/drone-person-detection.git

Cloning into 'drone-person-detection'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (3/3), done.


In [4]:
%cd drone-person-detection

/content/drone-person-detection


In [9]:
!git config --global user.name "Daniele Pavan"
!git config --global user.email "danipav95@gmail.com"

In [10]:
!git add .
!git commit -m "Aggiornamenti"
!git push origin main

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address
